1. [`10 pts`] What was the average time each driver spent at the pit stop for each race?
2. [`20 pts`] Rank the average time spent at the pit stop in order of who won each race
3. [`20 pts`] Insert the missing code (e.g: ALO for Alonso) for drivers based on the 'drivers' dataset
4. [`20 pts`] Who is the youngest and oldest driver for each race? Create a new column called “Age”
5. [`20 pts`] For a given race, which driver has the most wins and losses?
6. [`10 pts`] Continue exploring the data by answering your own question.[](url)

In [0]:
# import relevant functions
from pyspark.sql import functions as F
from pyspark.sql.functions import avg
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType

In [0]:
# import data
circuits = spark.read.csv('s3://columbia-gr5069-main/raw/circuits.csv', header=True) 
constructor_results = spark.read.csv('s3://columbia-gr5069-main/raw/constructor_results.csv', header=True) 
constructor_standings = spark.read.csv('s3://columbia-gr5069-main/raw/constructor_standings.csv', header=True) 
constructors = spark.read.csv('s3://columbia-gr5069-main/raw/constructors.csv', header=True) 
driver_standings = spark.read.csv('s3://columbia-gr5069-main/raw/driver_standings.csv', header=True) 
drivers = spark.read.csv('s3://columbia-gr5069-main/raw/drivers.csv', header=True) 
lap_times = spark.read.csv('s3://columbia-gr5069-main/raw/lap_times.csv', header=True) 
pit_stops = spark.read.csv('s3://columbia-gr5069-main/raw/pit_stops.csv', header=True) 
qualifying = spark.read.csv('s3://columbia-gr5069-main/raw/qualifying.csv', header=True) 
races = spark.read.csv('s3://columbia-gr5069-main/raw/races.csv', header=True) 
results = spark.read.csv('s3://columbia-gr5069-main/raw/results.csv', header=True) 
seasons = spark.read.csv('s3://columbia-gr5069-main/raw/seasons.csv', header=True) 
sprint_results = spark.read.csv('s3://columbia-gr5069-main/raw/sprint_results.csv', header=True) 
status = spark.read.csv('s3://columbia-gr5069-main/raw/status.csv', header=True) 
winequality = spark.read.csv('s3://columbia-gr5069-main/raw/winequality-red_main.csv', header=True) 

In [0]:
#1. [`10 pts`] What was the average time each driver spent at the pit stop for each race?

#create new column with full name of drivers 
drivers = drivers.withColumn("full_name", F.concat_ws(" ", drivers["forename"], drivers["surname"]))

#group by race id and driver id and calculate average duration of pit stop
avg_times = pit_stops.groupby(["raceId", "driverId"]).agg(avg("duration"))

#join with drivers_combined table to get names of drivers and their average duration of pit stop
avg_times = avg_times.join(drivers.select("driverId", "full_name"), "driverId", "left_outer")

display(avg_times)